In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from crewai_tools import tool
import yfinance as yf


class Tools:

    @tool("One month stock price history")
    def stock_price(ticker):
        """
        Useful to get a month's worth of stock price data as CSV.
        The input of this tool should a ticker, for example AAPL, NET, TSLA etc...
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Stock news URLs")
    def stock_news(ticker):
        """
        Useful to get URLs of news articles related to a stock.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker):
        """
        Useful to get the income statement of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker):
        """
        Useful to get the balance sheet of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker):
        """
        Useful to get insider transactions of a stock as CSV.
        The input to this tool should be a ticker, for example AAPL, NET
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

In [3]:
from crewai import Agent
from crewai_tools import ScrapeWebsiteTool


class Agents:

    def technical_analyst(self):
        return Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[
                Tools.stock_price,
            ],
        )

    def financial_analyst(self):
        return Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )

    def researcher(self):
        return Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                ScrapeWebsiteTool(),
            ],
        )

    def hedge_fund_manager(self):
        return Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )

In [4]:
from crewai import Task


class Tasks:

    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def finacial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )

In [5]:
from crewai import Crew
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
tasks = Tasks()

researcher = agents.researcher()
technical_analyst = agents.technical_analyst()
financial_analyst = agents.financial_analyst()
hedge_fund_manager = agents.hedge_fund_manager()

research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.finacial_analysis(financial_analyst)
recomment_task = tasks.investment_recommendation(
    hedge_fund_manager,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_manager,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recomment_task,
    ],
    verbose=2,
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=True,
)

result = crew.kickoff(
    inputs=dict(
        company="Salesforce",
    ),
)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Gather and analyze the latest news and market sentiment surrounding the stock of Salesforce. Provide a summary of the news and any notable shifts in market sentiment.


> Entering new CrewAgentExecutor chain...
To effectively gather and analyze the latest news and market sentiment surrounding Salesforce's stock, I will delegate this task to the Researcher. They are well-equipped to conduct thorough research and can provide a comprehensive summary of the latest developments. 

Action: Delegate work to co-worker  
Action Input: {"task": "Gather and analyze the latest news and market sentiment surrounding Salesforce's stock. Provide a detailed summary of the news and any notable shifts in market sentiment. Include insights on how these factors could impact the stock's performance.", "context": "Salesforce is a cloud-based software company known for its customer relationship management (CRM) service. The stock can be influenced 

2024-10-11 03:40:50,347 - 16488 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--8c8479665de86df451d05c1a9a17efb728881aa3d7540631f55effa6065873dc


Thought: I have gathered both the income statement and balance sheet for Salesforce. Next, I will retrieve the insider trading data to gain insights into management's confidence in the company through their buying and selling activities.

Action: Get insider transactions  
Action Input: {"ticker": "CRM"}  
 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.





2024-10-11 03:40:53,161 - 16488 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--c70d4badc1a423d6e8aba1be066fdb828f04fcff0732f681781afe85059a77b8


Thought: I need to gather Salesforce's financial statements and insider trading data to evaluate the company's financial health and performance effectively. First, I will obtain the income statement for Salesforce to analyze its revenue, earnings, and overall profitability.

Action: Company's income statement  
Action Input: {"ticker": "CRM"}  
 

,2024-01-31,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Tax Effect Of Unusual Items,-202400000.0,-224070000.0,69561357.70235,585900000.0,
Tax Rate For Calcs,0.16,0.21,0.057441,0.27,
Normalized EBITDA,11223000000.0,6711000000.0,2635000000.0,1131000000.0,
Total Unusual Items,-1265000000.0,-1067000000.0,1211000000.0,2170000000.0,
Total Unusual Items Excluding Goodwill,-1265000000.0,-1067000000.0,1211000000.0,2170000000.0,
Net Income From Continuing Operation Net Minority Interest,4136000000.0,208000000.0,1444000000.0,4072000000.0,
Reconciled Depreciation,3959000000.0,3786000000.0,3298000000.0,2846000000.0,
Reconciled Cost Of Revenue,8541000000.0

...

Thought: I need to gather comprehensive financial data on Salesforce to analyze its revenue growth, earnings performance, cash flow, balance sheet strength, insider trading data, and valuation. I will start by obtaining the company's income statement to evaluate revenue and earnings metrics.

Action: Company's income statement  
Action Input: {"ticker": "CRM"}  
 

,2024-01-31,2023-01-31,2022-01-31,2021-01-31,2020-01-31
Tax Effect Of Unusual Items,-202400000.0,-224070000.0,69561357.70235,585900000.0,
Tax Rate For Calcs,0.16,0.21,0.057441,0.27,
Normalized EBITDA,11223000000.0,6711000000.0,2635000000.0,1131000000.0,
Total Unusual Items,-1265000000.0,-1067000000.0,1211000000.0,2170000000.0,
Total Unusual Items Excluding Goodwill,-1265000000.0,-1067000000.0,1211000000.0,2170000000.0,
Net Income From Continuing Operation Net Minority Interest,4136000000.0,208000000.0,1444000000.0,4072000000.0,
Reconciled Depreciation,3959000000.0,3786000000.0,3298000000.0,2846000000.0,
Reconciled Cost Of Re

2024-10-11 03:44:09,630 - 16488 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--4b1a68b451049165b07b413b168176f91a84546f304aad20119347a091faa220


Thought: I have gathered the income statement and balance sheet for Salesforce, which provides insights into its revenue growth, earnings performance, and overall financial health. Next, I need to analyze insider trading data to assess market sentiment regarding the company.

Action: Get insider transactions  
Action Input: {"ticker": "CRM"}  
 

,Shares,Value,URL,Text,Insider,Position,Transaction,Start Date,Ownership
0,4200,495768.0,,Conversion of Exercise of derivative security at price 118.04 per share.,HARRIS G PARKER,Chief Technology Officer,,2024-10-08,D
1,4200,1211358.0,,Sale at price 285.03 - 292.12 per share.,HARRIS G PARKER,Chief Technology Officer,,2024-10-08,D
2,8427,1838856.0,,Conversion of Exercise of derivative security at price 218.21 per share.,MILLHAM BRIAN,President,,2024-10-02,D
3,1785,499800.0,,Sale at price 280.00 per share.,SCHMAIER ROBERT DAVID,President,,2024-10-02,D
4,8427,2359560.0,,Sale at price 280.00 per share.,MILLHAM BRIAN,President,,2024-10-02,D
5,4200,

2024-10-11 03:44:19,980 - 16488 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--4b1a68b451049165b07b413b168176f91a84546f304aad20119347a091faa220


Thought: I have gathered the income statement and balance sheet for Salesforce, which provides insights into its revenue growth, earnings performance, and overall financial health. Next, I need to analyze insider trading data to assess market sentiment regarding the company.

Action: Get insider transactions  
Action Input: {"ticker": "CRM"}  
 

,Shares,Value,URL,Text,Insider,Position,Transaction,Start Date,Ownership
0,4200,495768.0,,Conversion of Exercise of derivative security at price 118.04 per share.,HARRIS G PARKER,Chief Technology Officer,,2024-10-08,D
1,4200,1211358.0,,Sale at price 285.03 - 292.12 per share.,HARRIS G PARKER,Chief Technology Officer,,2024-10-08,D
2,8427,1838856.0,,Conversion of Exercise of derivative security at price 218.21 per share.,MILLHAM BRIAN,President,,2024-10-02,D
3,1785,499800.0,,Sale at price 280.00 per share.,SCHMAIER ROBERT DAVID,President,,2024-10-02,D
4,8427,2359560.0,,Sale at price 280.00 per share.,MILLHAM BRIAN,President,,2024-10-02,D
5,4200,